In [ ]:
%%cmd
pip install hca

In [67]:
#!/usr/bin/env python3

import hca, json


def count_cell_types(cell_type):
    count = 0
    query = {
        "query" : {
            "bool" : {
                "must" : [{
                    "match" : {
                        "metadata.files.biomaterial_json.biomaterials.context.organ.text" : "spleen"
                    }
                }, {
                    "match" : {
                        "metadata.files.biomaterial_json.schema_version" : "5.1.0"
                    }
                }, {
                    "range" : {
                        "metadata.files.biomaterial_json.biomaterials.content.total_estimated_cells" : {
                            "gt" : 0
                        }
                    }
                }]
            }
        }
    }
    
#    query = { "query" : { "bool" : { "must" : { "match" : { "*" : "*" } } } } }
    
#    query = { "query" : { "match" : { "search_score" : 1.0 } } }
    
    query = { "match" : { "search_score" : 1.0 } }
    
    # Iterate through all search results containing the chosen cell type
    for bundle_dict in hca.dss.DSSClient().post_search.iterate(es_query=query, replica='aws', output_format='raw'):
        
        # Check if the bundle is missing information. If so, skip to next iteration
        if not bundle_dict['metadata'].get('files'):
            continue
        
        print(json.dumps(bundle_dict, indent=4, sort_keys=True))
        print()
        
        # If the checks passed, then increment the count by the number of cells found in this file
        count += bundle_dict['metadata']['files']['biomaterial_json']['biomaterials'][0]['content']['total_estimated_cells']
        print(count)
        break
        
    return count


def main():
    organ_type = 'spleen'
    print('{} cell count: {}'.format(organ_type, count_cell_types(organ_type)))

    
main()

SwaggerAPIException: elasticsearch_bad_request: Invalid Elasticsearch query was received: TransportError(400, 'parsing_exception', 'Unknown key for a START_OBJECT in [match].') (HTTP 400). Details:
Traceback (most recent call last):
  File "/var/task/chalicelib/dss/api/search.py", line 47, in post
    page = _es_search_page(es_query, replica_enum, per_page, _scroll_id, output_format)
  File "/var/task/chalicelib/dss/api/search.py", line 119, in _es_search_page
    sort=sort
  File "/var/task/elasticsearch/client/utils.py", line 73, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/var/task/elasticsearch/client/__init__.py", line 632, in search
    doc_type, '_search'), params=params, body=body)
  File "/var/task/elasticsearch/transport.py", line 312, in perform_request
    status, headers, data = connection.perform_request(method, url, params, body, ignore=ignore, timeout=timeout)
  File "/var/task/elasticsearch/connection/http_requests.py", line 90, in perform_reque